In [1]:
# 1. Import libraries:
%matplotlib inline
import os
import quandl
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.optimize as spo
from scipy.stats import kurtosis, skew
import seaborn as sns
# from financial_data import *
import tensorflow as tf
mlp.style.use('seaborn')
quandl.save_key('HtwBLPt3k37yZHTvy15K')

/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/4242266090.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mlp.style.use('seaborn')


In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(sp_url, header=0)[0]
sp500.head()



,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
# Correct invalid dates:
sp500.loc[sp500[sp500['Date added']=='1983-11-30 (1957-03-04)'].index,'Date added'] = '1983-11-30'
sp500.loc[sp500[sp500['Date added']=='2001?'].index,'Date added'] = '2001-01-01'
# Filter firms that entered the index after December 2015:
sp500['Date added'] = pd.to_datetime(sp500['Date added'],format='%Y-%m-%d')
sp500 = sp500[sp500['Date added']<'2007-01-01']
print("The number of stocks in the universe is:", sp500.shape[0])

The number of stocks in the universe is: 241


In [4]:
n_stocks = 1
np.random.seed(1792)
universe_tickers = sp500['Symbol'].unique()
tickers = list(np.random.choice(universe_tickers,replace=False,size=n_stocks))
print(tickers)
# sp500[sp500['Symbol'].isin(portfolio_tickers)]

['DIS']


In [5]:
# tickers = ['AAPL', 'GOOGL', 'MSFT','SPY']
# start_date='2007-01-01'
# end_date='2022-12-31'
# columns = ['Adj Close', 'Volume']
# # Download data
# data = yf.download(tickers, start=start_date, end=end_date)[columns]

# # 'data' will be a Pandas DataFrame containing the historical data for the specified tickers
# print(data)


def download_data(tickers, start_date, end_date, columns):
    data = yf.download(tickers, start=start_date, end=end_date)
    data=data[columns]
    return data

# Define the tickers, start date, end date, and columns you want to download
# tickers = ['AAPL', 'GOOGL', 'MSFT']
start_date = '2007-01-01'
end_date = '2022-12-31'
columns = [ 'Adj Close', 'Volume','Open','High','Low']

# Download the data
data = download_data(tickers, start_date, end_date, columns)
print(data)


def one_lvl_colnames(df,cols,tickers):
    """This function changes a multi-level column indexation into a one level
    column indexation

    Inputs:
    -------
    df (pandas Dataframe): dataframe with the columns whose indexation will be 
        flattened.
    tickers (list|string): list/string with the tickers (s) in the data frame df.
    cols (list|string): list/string with the name of the columns (e.g. 'Adj Close',
        'High', 'Close', etc.) that are in the dataframe df.
    
    Ouputs:
    -------
    df (pandas Dataframe): dataframe with the same information as df, but 
        with one level of indexation.
    """
    
    
    print("the dataframe before the columns were renamed")
#     print(df.columns)
    df_not_renamed=df.copy()
#     df_not_renamed.drop("Open",inplace=True)
    print(df_not_renamed.columns)
    # Define important variables:
    if isinstance(tickers, str):
        tickers = [tickers]
    if isinstance(cols, str):
        
        cols = [cols]
        print(cols)

    # For multi-level column indexing:
    if isinstance(df.columns.values[0], tuple):

        # Define important varibles
        columns = df.columns.values
        new_cols = []

        # Itarate through the multi-level column names and flatten them:
        for col in columns:
            temp = []
            for name in col:
                if name != '':
                    temp.append(name)
            new_temp = '_'.join(temp)
            new_cols.append(new_temp)
        
        # Change the column names:
        df.columns = new_cols
        print(df.columns)
    
    # For uni-level colum indexing:
    elif isinstance(df.columns.values[0], str):
        
        # Define new names:
        col_names = [column+'_'+ticker for column in cols\
                     for ticker in tickers]
        df.columns = col_names
    
    print("the dataframe after the columns have been renamed") 
#     print(df)

    return df

df2=pd.DataFrame(one_lvl_colnames(data,cols=columns,tickers=tickers))
df2_columns=list(df2.columns)
df2.columns = [col.replace('Adj Close', 'Close') for col in df2_columns]
df2.fillna(method='ffill',inplace=True)
df2.fillna(method='bfill',inplace=True)
df2

[*********************100%%**********************]  1 of 1 completed
            Adj Close    Volume       Open       High        Low
Date                                                            
2007-01-03  28.317097  13562595  33.748165  34.073711  33.531136
2007-01-04  28.540653   9806285  33.738300  34.083576  33.708706
2007-01-05  28.308819  10551445  33.807354  33.975060  33.531136
2007-01-08  28.565487   9479676  33.728436  34.162495  33.610054
2007-01-09  28.524090  11588444  34.034248  34.221684  33.491676
...               ...       ...        ...        ...        ...
2022-12-23  88.010002  11171600  86.059998  88.070000  85.769997
2022-12-27  86.370003  11561400  87.419998  87.940002  85.959999
2022-12-28  84.169998  12399500  86.080002  86.690002  84.070000
2022-12-29  87.180000  13045100  85.250000  88.239998  84.970001
2022-12-30  86.879997  23231000  85.730003  87.120003  85.230003

[4028 rows x 5 columns]
the dataframe before the columns were renamed
Index(['Adj Clo

,Close_DIS,Volume_DIS,Open_DIS,High_DIS,Low_DIS
Date,,,,,
2007-01-03,28.317097,13562595,33.748165,34.073711,33.531136
2007-01-04,28.540653,9806285,33.738300,34.083576,33.708706
2007-01-05,28.308819,10551445,33.807354,33.975060,33.531136
2007-01-08,28.565487,9479676,33.728436,34.162495,33.610054
2007-01-09,28.524090,11588444,34.034248,34.221684,33.491676
...,...,...,...,...,...
2022-12-23,88.010002,11171600,86.059998,88.070000,85.769997
2022-12-27,86.370003,11561400,87.419998,87.940002,85.959999
2022-12-28,84.169998,12399500,86.080002,86.690002,84.070000


In [6]:
import pandas as pd

# Assuming df is your dataframe
# If it's not loaded yet, you should load it using pd.read_csv() or any other method

# Get a list of unique tickers
tickers = df2.columns.str.split('_').str[1].unique()

# Create a dictionary to store information for each ticker
ticker_data = {}

# Loop through each ticker and extract the information
for ticker in tickers:
    ticker_info = {}
    
    # Iterate through the columns and filter based on the current ticker
    for column in df2.columns:
        if ticker in column:
            ticker_info[column] = df2[column]
    
    # Store the ticker information in the dictionary
    ticker_data[ticker] = ticker_info

# Now, you can access information for each ticker using ticker_data dictionary
# For example, to get the Adj Close data for BIIB, you can use:
biib = pd.DataFrame(ticker_data['DIS'])
biib.columns=biib.columns.str.split('_').str[0].unique()

In [7]:
biib
from stockstats import StockDataFrame

stock_df_biib = StockDataFrame.retype(biib)
stock_df_biib

stock_df_biib.init_all()

In [8]:
stock_df_biib

columns = list(stock_df_biib.columns)
# columns

In [9]:
stock_df_biib.shape

(4028, 74)

In [10]:
[stock_df_biib.columns]

[Index(['close', 'volume', 'open', 'high', 'low', 'rate', 'middle', 'tp',
        'boll', 'boll_ub', 'boll_lb', 'macd', 'macds', 'macdh', 'pvo', 'pvos',
        'pvoh', 'ppo', 'ppos', 'ppoh', 'qqe', 'qqel', 'qqes', 'cr', 'cr-ma1',
        'cr-ma2', 'cr-ma3', 'tr', 'dx', 'adx', 'adxr', 'log-ret', 'wt1', 'wt2',
        'supertrend_ub', 'supertrend_lb', 'supertrend', 'bop', 'cti', 'eribull',
        'eribear', 'rvgi', 'rvgis', 'kst', 'ao', 'aroon', 'atr', 'cci',
        'change', 'chop', 'cmo', 'coppock', 'dma', 'ichimoku', 'inertia', 'ftr',
        'kama', 'kdjk', 'kdjd', 'kdjj', 'ker', 'mfi', 'ndi', 'pdi', 'pgo',
        'psl', 'rsi', 'rsv', 'stochrsi', 'tema', 'trix', 'wr', 'vr', 'vwma'],
       dtype='object')]

In [11]:
# momentum=['ao','bop', 'cci', 'change', 'chop', 'cmo', 'coppock', 'eribear', 'eribull',  'inertia', 
#           'kdjd', 'kdjj', 'kdjk',  'kst', 'macd', 'macdh', 'macds', 'pgo', 'ppo', 'ppoh', 'ppos', 'psl',
#         'pvo', 'pvoh', 'pvos', 'qqe', 'qqel', 'qqes', 'rate', 'rsi', 'rsv',
#         'rvgi', 'rvgis', 'stochrsi','trix']
# trend=['adx', 'adxr','aroon', 'cti', 'dx','ndi','pdi','wt1','wt2']
# overlap=['ichimoku','kama', 'supertrend', 'supertrend_lb',
#         'supertrend_ub','vwma','tema']
# volatility=[ 'atr', 'boll', 'boll_lb', 'boll_ub',]
# rem_ind=['cr','cr-ma1', 'cr-ma2', 'cr-ma3', 'dma','ftr','ker', 'log-ret','mfi',
#         'middle','tp','tr','vr','wr']

In [12]:
momentum=['ao','bop', 'cci', 'change', 'chop', 'cmo', 'coppock', 'eribear', 'eribull',  'inertia', 
          'kdjd', 'kdjj', 'kdjk',  'kst', 'macd', 'macdh', 'macds', 'pgo', 'ppo', 'ppoh', 'ppos', 'psl',
        'pvo', 'pvoh', 'pvos', 'qqe', 'qqel', 'qqes', 'rate', 'rsi', 'rsv',
        'rvgi', 'rvgis', 'stochrsi','trix']
trend=['adx', 'adxr','aroon', 'cti', 'dx','ndi','pdi','wt1','wt2']
overlap=['ichimoku','kama', 'supertrend', 'supertrend_lb',
        'supertrend_ub','vwma','tema']
volatility=[ 'atr', 'boll', 'boll_lb', 'boll_ub',]
rem_ind=['cr','cr-ma1', 'cr-ma2', 'cr-ma3', 'dma','ftr','ker', 'log-ret','mfi',
        'middle','tp','tr','vr','wr']
base_col=['open','high','close','volume','low']
target=['close']

In [13]:
len(momentum)+len(trend)+len(overlap)+len(volatility)+len(rem_ind)

69

In [14]:
stock_df_biib
stock_df_biib=stock_df_biib[sorted(stock_df_biib.columns)]

In [15]:
pd.set_option('display.max_columns', 100)

In [16]:
ind_types={'momentum': ['ao','bop', 'cci', 'change', 'chop', 'cmo', 'coppock', 'eribear', 'eribull',  'inertia', 
          'kdjd', 'kdjj', 'kdjk',  'kst', 'macd', 'macdh', 'macds', 'pgo', 'ppo', 'ppoh', 'ppos', 'psl',
        'pvo', 'pvoh', 'pvos', 'qqe', 'qqel', 'qqes', 'rate', 'rsi', 'rsv',
        'rvgi', 'rvgis', 'stochrsi','trix'],
 'volatility': ['atr', 'boll', 'boll_lb', 'boll_ub','tr'],
 'rem_ind': ['cr','cr-ma1', 'cr-ma2', 'cr-ma3', 'dma','ftr','ker', 'log-ret','mfi',
        'middle','tp','vr','wr'],
 'overlap': ['ichimoku','kama', 'supertrend', 'supertrend_lb',
        'supertrend_ub','vwma','tema'],
          'trend':['adx', 'adxr','aroon', 'cti', 'dx','ndi','pdi','wt1','wt2']}
for ind_type,metrics in ind_types.items():
    print(ind_type, 'corresponds to', metrics)

momentum corresponds to ['ao', 'bop', 'cci', 'change', 'chop', 'cmo', 'coppock', 'eribear', 'eribull', 'inertia', 'kdjd', 'kdjj', 'kdjk', 'kst', 'macd', 'macdh', 'macds', 'pgo', 'ppo', 'ppoh', 'ppos', 'psl', 'pvo', 'pvoh', 'pvos', 'qqe', 'qqel', 'qqes', 'rate', 'rsi', 'rsv', 'rvgi', 'rvgis', 'stochrsi', 'trix']
volatility corresponds to ['atr', 'boll', 'boll_lb', 'boll_ub', 'tr']
rem_ind corresponds to ['cr', 'cr-ma1', 'cr-ma2', 'cr-ma3', 'dma', 'ftr', 'ker', 'log-ret', 'mfi', 'middle', 'tp', 'vr', 'wr']
overlap corresponds to ['ichimoku', 'kama', 'supertrend', 'supertrend_lb', 'supertrend_ub', 'vwma', 'tema']
trend corresponds to ['adx', 'adxr', 'aroon', 'cti', 'dx', 'ndi', 'pdi', 'wt1', 'wt2']


In [17]:
import pandas as pd

# Assuming df is your dataframe with features and target variable
# If it's not loaded yet, you should load it using pd.read_csv() or any other method
stock_df_biib_corr_temp=stock_df_biib.copy()
for ind_type,metrics in ind_types.items():
    print(ind_type, 'corresponds to', metrics)
    # Calculate the correlation matrix
    stock_df_biib_corr=stock_df_biib_corr_temp[metrics+target]
    stock_df_biib_corr.dropna(inplace=True)
    correlation_matrix = stock_df_biib_corr.corr()

    # Stack the correlation matrix to create a DataFrame
    correlation_df = correlation_matrix.stack().reset_index()
    correlation_df.columns = ['Feature1', 'Feature2', 'Correlation']
    correlation_df = correlation_df[(correlation_df['Feature1'] != correlation_df['Feature2']) 
                                    & (correlation_df['Feature1'] < correlation_df['Feature2'])]

    correlation_df=pd.DataFrame(correlation_df.sort_values(by='Correlation', ascending=False).reset_index(drop=True))
#     correlation_df=correlation_df[correlation_df['Correlation']]
    # Print the resulting DataFrame
    print("DataFrame from Pairwise Correlation:")
    # print(correlation_df.loc[150:200])
    print(correlation_df.head(100))
    


# Print the sorted correlations
# print(pd.DataFrame(correlation_matrix))

momentum corresponds to ['ao', 'bop', 'cci', 'change', 'chop', 'cmo', 'coppock', 'eribear', 'eribull', 'inertia', 'kdjd', 'kdjj', 'kdjk', 'kst', 'macd', 'macdh', 'macds', 'pgo', 'ppo', 'ppoh', 'ppos', 'psl', 'pvo', 'pvoh', 'pvos', 'qqe', 'qqel', 'qqes', 'rate', 'rsi', 'rsv', 'rvgi', 'rvgis', 'stochrsi', 'trix']
DataFrame from Pairwise Correlation:
   Feature1 Feature2  Correlation
0    change     rate     1.000000
1      rvgi    rvgis     0.993031
2       kst     trix     0.984103
3      kdjd     kdjk     0.982538
4      kdjj      rsv     0.981530
..      ...      ...          ...
95      cmo      ppo     0.650088
96    close      rsv     0.647380
97      kst      rsi     0.645589
98      cmo     qqel     0.637353
99      cmo      pgo     0.636902

[100 rows x 3 columns]
volatility corresponds to ['atr', 'boll', 'boll_lb', 'boll_ub', 'tr']
DataFrame from Pairwise Correlation:
   Feature1 Feature2  Correlation
0      boll  boll_ub     0.997858
1      boll  boll_lb     0.997353
2      bo

In [18]:
import pandas as pd

# Assuming df is your dataframe with features and target variable
# If it's not loaded yet, you should load it using pd.read_csv() or any other method
stock_df_biib_corr_temp=stock_df_biib.copy()
for ind_type,metrics in ind_types.items():
    print(ind_type, 'corresponds to', metrics)
    # Calculate the correlation matrix
    
    stock_df_biib_corr=stock_df_biib_corr_temp[metrics+target]
    stock_df_biib_corr.dropna(inplace=True)
    correlation_matrix = stock_df_biib_corr.corr()

    # Stack the correlation matrix to create a DataFrame
    correlation_df = correlation_matrix.stack().reset_index()
    correlation_df.columns = ['Feature1', 'Feature2', 'Correlation']
    correlation_df = correlation_df[(correlation_df['Feature1'] != correlation_df['Feature2']) 
                                    & (correlation_df['Feature1'] < correlation_df['Feature2'])]

    correlation_df=pd.DataFrame(correlation_df.sort_values(by='Correlation', ascending=False).reset_index(drop=True))
#     correlation_df=correlation_df[correlation_df['Correlation']]
    # Print the resulting DataFrame
#     print("DataFrame from Pairwise Correlation:")
    # print(correlation_df.loc[150:200])
#     print(correlation_df.head(100))
#     print("********** ***********************")
    # Assuming 'Target_Variable' is the name of your target variable column
    target_correlation = correlation_matrix['close'].abs()

    # Sort the correlations in descending order to see the most correlated features
    sorted_correlations = target_correlation.sort_values(ascending=False)

    # Print the sorted correlations
    print(pd.DataFrame(sorted_correlations)[1:])



# Print the sorted correlations
# print(pd.DataFrame(correlation_matrix))

momentum corresponds to ['ao', 'bop', 'cci', 'change', 'chop', 'cmo', 'coppock', 'eribear', 'eribull', 'inertia', 'kdjd', 'kdjj', 'kdjk', 'kst', 'macd', 'macdh', 'macds', 'pgo', 'ppo', 'ppoh', 'ppos', 'psl', 'pvo', 'pvoh', 'pvos', 'qqe', 'qqel', 'qqes', 'rate', 'rsi', 'rsv', 'rvgi', 'rvgis', 'stochrsi', 'trix']
             close
rvgi      0.762613
rvgis     0.761192
chop      0.750399
inertia   0.743020
kdjd      0.701302
kdjk      0.683049
bop       0.651099
rsv       0.647380
kdjj      0.596657
eribear   0.428955
eribull   0.246415
macds     0.086887
macd      0.079935
ao        0.060090
ppos      0.036049
cmo       0.032476
ppo       0.032435
kst       0.030467
trix      0.028287
cci       0.027190
rsi       0.026717
pgo       0.024013
stochrsi  0.022903
pvos      0.019524
coppock   0.017669
pvo       0.014823
macdh     0.008436
ppoh      0.005658
psl       0.004000
qqel      0.003832
rate      0.003739
change    0.003739
qqes      0.002969
qqe       0.001857
pvoh      0.000703
vol

In [19]:
# stock_df_biib
# df_copy=stock_df_biib.copy()
# df_copy=df_copy.iloc[14:-1]
# features_to_shift=['close']
# df_copy[features_to_shift] = df_copy[features_to_shift].apply(lambda x: x.shift(-1))



In [20]:
# df_copy.isnull().sum().sort_values(ascending=False)

In [21]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 20)

# vif_data=pd.DataFrame(vif_data)
# vif_data.head(70)

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
def check_var_importance(df,metrics):
    # Separate features (X) and target (y)
    # AAPL_df.drop(columns=["AAPL_cr"],inplace=True)
    df.ffill(inplace=True)
    df.bfill(inplace=True)
    print("**************************")
#     print(df.isnull())

    X = df  # Use relevant columns
    y = df['close']  # Target variable is 'Close'

    # Standardize features (optional but often recommended for neural networks)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Create sequences of length 5 for X and corresponding y for training
    X_train, y_train = [], []

    for i in range(5, len(X)):
        X_train.append(X[i-5:i])
        y_train.append(y[i])

    X_train, y_train = np.array(X_train), np.array(y_train)
#     print(X_train)
#     print("***************")
#     print(y_train)

    # Step 2: Check Variable Importance
    # Use RandomForestRegressor to determine importance
    rf = RandomForestRegressor(n_estimators=10)
    rf.fit(X_train.reshape(-1, 5 * X.shape[1]), y_train)

    # Get feature importances along with their corresponding feature names
    feature_importances = rf.feature_importances_
    feature_names = metrics
#     feature_names=feature_names.remove('close')
#     print(feature_names)

    # Combine feature names with their importances
    feature_importance_info = list(zip(feature_names, feature_importances))
    print(feature_importance_info)

    # Sort the list by importance (in descending order)
    feature_importance_info = sorted(feature_importance_info, key=lambda x: x[1], reverse=True)

    return pd.DataFrame(feature_importance_info,columns=['Features','Relative_Importance'])

    

In [23]:
ind_types={'momentum': ['ao','bop', 'cci', 'change', 'chop', 'cmo', 'coppock', 'eribear', 'eribull',  'inertia', 
          'kdjd', 'kdjj', 'kdjk',  'kst', 'macd', 'macdh', 'macds', 'pgo', 'ppo', 'ppoh', 'ppos', 'psl',
        'pvo', 'pvoh', 'pvos', 'qqe', 'qqel', 'qqes', 'rsi', 'rsv',
        'rvgi', 'rvgis', 'stochrsi','trix'],
 'volatility': ['atr', 'boll', 'boll_lb', 'boll_ub','tr'],
 'rem_ind': [ 'dma','ftr','ker', 'log-ret','mfi',
        'middle','vr','wr'],
 'overlap': ['ichimoku','kama', 'supertrend', 'supertrend_lb',
        'supertrend_ub','vwma','tema'],
          'trend':['adx', 'adxr','aroon', 'cti', 'dx','ndi','pdi','wt1','wt2']}
for ind_type,metrics in ind_types.items():
    print(ind_type, 'corresponds to', metrics)

momentum corresponds to ['ao', 'bop', 'cci', 'change', 'chop', 'cmo', 'coppock', 'eribear', 'eribull', 'inertia', 'kdjd', 'kdjj', 'kdjk', 'kst', 'macd', 'macdh', 'macds', 'pgo', 'ppo', 'ppoh', 'ppos', 'psl', 'pvo', 'pvoh', 'pvos', 'qqe', 'qqel', 'qqes', 'rsi', 'rsv', 'rvgi', 'rvgis', 'stochrsi', 'trix']
volatility corresponds to ['atr', 'boll', 'boll_lb', 'boll_ub', 'tr']
rem_ind corresponds to ['dma', 'ftr', 'ker', 'log-ret', 'mfi', 'middle', 'vr', 'wr']
overlap corresponds to ['ichimoku', 'kama', 'supertrend', 'supertrend_lb', 'supertrend_ub', 'vwma', 'tema']
trend corresponds to ['adx', 'adxr', 'aroon', 'cti', 'dx', 'ndi', 'pdi', 'wt1', 'wt2']


In [24]:
final_df2=stock_df_biib.copy()
final_df2=final_df2.drop(columns=['cr', 'cr-ma1', 'cr-ma2', 'cr-ma3'])
feat_importance_all=pd.DataFrame()
for ind_type,metrics in ind_types.items():
    print(ind_type, 'corresponds to', metrics)
#     print(matching_columns)
#     matching_columns.append([])
    # # Print matching columns
    # for col in matching_columns:
    #     print(col)

    temp_df=final_df2[metrics+target]
    temp_df.dropna(inplace=True)
    feat_importance_temp=check_var_importance(temp_df,metrics)
    print(feat_importance_temp)


momentum corresponds to ['ao', 'bop', 'cci', 'change', 'chop', 'cmo', 'coppock', 'eribear', 'eribull', 'inertia', 'kdjd', 'kdjj', 'kdjk', 'kst', 'macd', 'macdh', 'macds', 'pgo', 'ppo', 'ppoh', 'ppos', 'psl', 'pvo', 'pvoh', 'pvos', 'qqe', 'qqel', 'qqes', 'rsi', 'rsv', 'rvgi', 'rvgis', 'stochrsi', 'trix']
**************************


/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3283412936.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.dropna(inplace=True)
/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3661443465.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ffill(inplace=True)
/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3661443465.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.bf

[('ao', 2.358262496288406e-06), ('bop', 1.1778625466627868e-05), ('cci', 2.9746978283123534e-06), ('change', 1.4152454188723884e-05), ('chop', 1.9045238280400807e-05), ('cmo', 3.0904885681922302e-06), ('coppock', 6.498700484304141e-06), ('eribear', 5.381091556624697e-06), ('eribull', 1.0038452285218629e-05), ('inertia', 9.686233167066038e-06), ('kdjd', 1.973847188041191e-05), ('kdjj', 3.6557522786413955e-06), ('kdjk', 1.3086114653585208e-05), ('kst', 1.894942557380939e-05), ('macd', 1.1198219963022758e-06), ('macdh', 8.476632154784958e-06), ('macds', 4.229841227778235e-06), ('pgo', 3.6732068241184644e-06), ('ppo', 1.93611538859796e-06), ('ppoh', 2.6009744106426616e-06), ('ppos', 4.064946623623301e-06), ('psl', 1.7788780335428728e-06), ('pvo', 9.986036274811606e-06), ('pvoh', 8.89836345567876e-06), ('pvos', 8.544305722056245e-06), ('qqe', 4.339021285481309e-06), ('qqel', 6.73905678549467e-07), ('qqes', 1.7733363203295268e-06), ('rsi', 6.492098977618591e-06), ('rsv', 7.1267434314383815e-

/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3283412936.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.dropna(inplace=True)
/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3661443465.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ffill(inplace=True)
/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3661443465.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.bf

[('atr', 6.117943072638646e-05), ('boll', 0.00016511623232114166), ('boll_lb', 2.5858500857036443e-05), ('boll_ub', 2.3882254317335816e-05), ('tr', 7.414701716651031e-05)]
  Features  Relative_Importance
0     boll             0.000165
1       tr             0.000074
2      atr             0.000061
3  boll_lb             0.000026
4  boll_ub             0.000024
rem_ind corresponds to ['dma', 'ftr', 'ker', 'log-ret', 'mfi', 'middle', 'vr', 'wr']
**************************


...

[('dma', 4.365302333976975e-05), ('ftr', 2.4443218344234165e-05), ('ker', 2.749718589190679e-05), ('log-ret', 4.093692859503214e-05), ('mfi', 2.6574196908652442e-05), ('middle', 0.0062856331828720635), ('vr', 2.531286386428274e-05), ('wr', 1.9891452860656072e-05)]
  Features  Relative_Importance
0   middle             0.006286
1      dma             0.000044
2  log-ret             0.000041
3      ker             0.000027
4      mfi             0.000027
5       vr             0.000025
6      ftr             0.000024
7       wr             0.000020
overlap corresponds to ['ichimoku', 'kama', 'supertrend', 'supertrend_lb', 'supertrend_ub', 'vwma', 'tema']
**************************


/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3283412936.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.dropna(inplace=True)
/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3661443465.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ffill(inplace=True)
/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3661443465.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.bf

[('ichimoku', 3.261880335155846e-05), ('kama', 2.2275200539313883e-05), ('supertrend', 1.0440603291377536e-05), ('supertrend_lb', 2.2582720906650187e-05), ('supertrend_ub', 1.5953365593938472e-05), ('vwma', 0.000148065672301319), ('tema', 3.302113046188897e-05)]
        Features  Relative_Importance
0           vwma             0.000148
1           tema             0.000033
2       ichimoku             0.000033
3  supertrend_lb             0.000023
4           kama             0.000022
5  supertrend_ub             0.000016
6     supertrend             0.000010
trend corresponds to ['adx', 'adxr', 'aroon', 'cti', 'dx', 'ndi', 'pdi', 'wt1', 'wt2']
**************************


/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3283412936.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.dropna(inplace=True)
/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3661443465.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ffill(inplace=True)
/var/folders/dk/mxqr91nj7135zkb7ptp3mq5c0000gn/T/ipykernel_8149/3661443465.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.bf

[('adx', 1.4172174354168916e-05), ('adxr', 2.6925084276358218e-05), ('aroon', 9.361347533747577e-05), ('cti', 1.964549892626948e-05), ('dx', 2.2745314683001618e-05), ('ndi', 2.8223202304687397e-05), ('pdi', 2.2764224302707804e-05), ('wt1', 2.1309793467707593e-05), ('wt2', 3.341570940245773e-05)]
  Features  Relative_Importance
0    aroon             0.000094
1      wt2             0.000033
2      ndi             0.000028
3     adxr             0.000027
4      pdi             0.000023
5       dx             0.000023
6      wt1             0.000021
7      cti             0.000020
8      adx             0.000014
